In [ ]:
import os
import random
import time
from itertools import permutations

DATA_DIR = "./Data"  # Adjust if your input files are in a different directory

def read_input(file_name):
    """Reads and parses the input file."""
    file_path = os.path.join(DATA_DIR, file_name)
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    # First line contains the number of paintings
    n = int(lines[0].strip())
    paintings = []
    
    # Process each painting line
    for i, line in enumerate(lines[1:], start=0):
        parts = line.strip().split()
        orientation = parts[0]
        num_tags = int(parts[1])
        tags = set(parts[2:])  # Use a set to ensure uniqueness of tags
        paintings.append({"ID": i, "Orientation": orientation, "Tags": tags})
    
    return n, paintings


In [2]:
def create_frameglasses(paintings):
    """
    Generates frameglasses from paintings:
    - Each Landscape (L) is in its own frameglass.
    - Each frameglass for Portraits (P) combines exactly two paintings.
    - Tags in a frameglass are unique.
    """
    landscapes = [p for p in paintings if p["Orientation"] == "L"]
    portraits = [p for p in paintings if p["Orientation"] == "P"]

    frameglasses = []

    # Add each Landscape as its own frameglass
    for landscape in landscapes:
        frameglasses.append({
            "Paintings": [landscape["ID"]],  # Only one Landscape painting per frameglass
            "Tags": landscape["Tags"]  # Tags remain as-is
        })

    # Group every two Portraits together
    for i in range(0, len(portraits), 2):
        if i + 1 < len(portraits):  # Ensure there is a pair
            combined_tags = portraits[i]["Tags"] | portraits[i + 1]["Tags"]  # Combine unique tags
            frameglasses.append({
                "Paintings": [portraits[i]["ID"], portraits[i + 1]["ID"]],
                "Tags": combined_tags
            })
        else:
            # If there is an odd number of Portraits, leave the last one ungrouped (optional)
            frameglasses.append({
                "Paintings": [portraits[i]["ID"]],
                "Tags": portraits[i]["Tags"]
            })

    return frameglasses


In [3]:
def calculate_score(frameglasses):
    """Calculates the Global Robotic Satisfaction."""
    score = 0
    for i in range(len(frameglasses) - 1):
        tags1 = frameglasses[i]["Tags"]
        tags2 = frameglasses[i + 1]["Tags"]

        # Calculate Local Robotic Satisfaction
        common_tags = len(tags1 & tags2)
        tags_in_f1_not_in_f2 = len(tags1 - tags2)
        tags_in_f2_not_in_f1 = len(tags2 - tags1)
        local_score = min(common_tags, tags_in_f1_not_in_f2, tags_in_f2_not_in_f1)
        score += local_score

    return score


In [4]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import numpy as np

def compute_pair(args):
    try:
        i, j, frameglasses = args
        tags_i = frameglasses[i]["Tags"]
        tags_j = frameglasses[j]["Tags"]

        common_tags = len(tags_i & tags_j)
        tags_in_i_not_in_j = len(tags_i - tags_j)
        tags_in_j_not_in_i = len(tags_j - tags_i)

        min_form = min(common_tags, tags_in_i_not_in_j, tags_in_j_not_in_i)
        return i, j, min_form
    except Exception as e:
        print(f"Error in worker process: {e}")
        raise

def calculate_min_form_tags_fully_parallel(frameglasses):
    n = len(frameglasses)
    min_form_tags_matrix = np.zeros((n, n))

    pair_indices = [(i, j, frameglasses) for i in range(n) for j in range(i + 1, n)]

    try:
        with ProcessPoolExecutor(max_workers=4) as executor:
            results = executor.map(compute_pair, pair_indices)
    except Exception as e:
        print("Falling back to ThreadPoolExecutor due to ProcessPoolExecutor failure.")
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(compute_pair, pair_indices)

    for i, j, min_form in results:
        min_form_tags_matrix[i, j] = min_form
        min_form_tags_matrix[j, i] = min_form

    return min_form_tags_matrix


In [5]:
def compute_pair(args):
    try:
        i, j, frameglasses = args
        tags_i = frameglasses[i]["Tags"]
        tags_j = frameglasses[j]["Tags"]

        common_tags = len(tags_i & tags_j)
        tags_in_i_not_in_j = len(tags_i - tags_j)
        tags_in_j_not_in_i = len(tags_j - tags_i)

        min_form = min(common_tags, tags_in_i_not_in_j, tags_in_j_not_in_i)
        return i, j, min_form
    except Exception as e:
        print(f"Error in worker process: {e}")
        raise
def compute_pair(args):
    try:
        i, j, frameglasses = args
        tags_i = frameglasses[i]["Tags"]
        tags_j = frameglasses[j]["Tags"]

        common_tags = len(tags_i & tags_j)
        tags_in_i_not_in_j = len(tags_i - tags_j)
        tags_in_j_not_in_i = len(tags_j - tags_i)

        min_form = min(common_tags, tags_in_i_not_in_j, tags_in_j_not_in_i)
        return i, j, min_form
    except Exception as e:
        print(f"Error in worker process: {e}")
        raise


In [6]:
def write_output(frameglasses, output_file_name):
    """Writes the frameglasses to an output file."""
    with open(output_file_name, "w") as file:
        file.write(f"{len(frameglasses)}\n")
        for fg in frameglasses:
            file.write(" ".join(map(str, fg["Paintings"])) + "\n")


In [7]:
def measure_execution_time(func, *args):
    """Measures the execution time of a function."""
    start_time = time.time()
    result = func(*args)
    elapsed_time = time.time() - start_time
    print(f"{func.__name__} executed in {elapsed_time:.4f} seconds")
    return result, elapsed_time


In [8]:
import os
import logging
from multiprocessing import Process, Queue

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_file(input_file, queue=None):
    """Processes a single file and optionally puts the result in a queue."""
    try:
        logging.info(f"Processing {input_file}...")

        # Step 1: Parse input
        n, paintings = read_input(input_file)
        logging.debug(f"Parsed input: n={n}, paintings={paintings}")

        # Step 2: Create frameglasses
        frameglasses = create_frameglasses(paintings)
        logging.debug(f"Created frameglasses: {frameglasses}")

        # Step 3: Calculate score
        score = calculate_score(frameglasses)
        logging.info(f"Score for {input_file}: {score}")

        # Step 4: Write the sorted frameglasses to an output file
        # Fix the output path to avoid duplicating the DATA_DIR
        output_file_name = os.path.normpath(os.path.join(DATA_DIR, f"{os.path.splitext(os.path.basename(input_file))[0]}_sorted.txt"))
        write_output(frameglasses, output_file_name)
        logging.info(f"Wrote sorted frameglasses to {output_file_name}")

        if queue:
            queue.put(score)
        return score
    except Exception as e:
        logging.error(f"Error processing {input_file}: {e}")
        if queue:
            queue.put(0)
        return 0

def main():
    # Correctly construct the list of input files
    input_files = [os.path.normpath(os.path.join(DATA_DIR, f)) for f in os.listdir(DATA_DIR) if f.endswith(".txt")]
    
    # Separate files into parallel and sequential processing
    parallel_files = [f for f in input_files if os.path.basename(f).startswith(("0", "10"))]
    sequential_files = [f for f in input_files if f not in parallel_files]
    
    total_score = 0
    queue = Queue()
    processes = []

    # Process files in parallel
    for input_file in parallel_files:
        p = Process(target=process_file, args=(input_file, queue))
        processes.append(p)
        p.start()

    # Wait for all parallel processes to finish
    for p in processes:
        p.join()

    # Collect results from parallel processes
    while not queue.empty():
        total_score += queue.get()

    # Process files sequentially
    for input_file in sequential_files:
        total_score += process_file(input_file)

    logging.info(f"Total Global Score: {total_score}")

    print(input_files)

if __name__ == "__main__":
    main()


2024-11-20 15:00:18,168 - INFO - Processing Data\110_oily_portraits.txt...
2024-11-20 15:00:18,169 - ERROR - Error processing Data\110_oily_portraits.txt: [Errno 2] No such file or directory: './Data\\Data\\110_oily_portraits.txt'
2024-11-20 15:00:18,169 - INFO - Processing Data\11_randomizing_paintings.txt...
2024-11-20 15:00:18,171 - ERROR - Error processing Data\11_randomizing_paintings.txt: [Errno 2] No such file or directory: './Data\\Data\\11_randomizing_paintings.txt'
2024-11-20 15:00:18,171 - INFO - Processing Data\1_binary_landscapes.txt...
2024-11-20 15:00:18,172 - ERROR - Error processing Data\1_binary_landscapes.txt: [Errno 2] No such file or directory: './Data\\Data\\1_binary_landscapes.txt'
2024-11-20 15:00:18,173 - INFO - Total Global Score: 0


['Data\\0_example.txt', 'Data\\10_computable_moments.txt', 'Data\\110_oily_portraits.txt', 'Data\\11_randomizing_paintings.txt', 'Data\\1_binary_landscapes.txt']
